In [1]:
# Python version: 3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) [GCC 7.2.0]
# Results were generated with numpy v1.15.1, pandas v0.23.4 and scikit-learn v0.19.1
# random seed = 42
# Training dataset filename = iris.csv
# Pickle filename = model_5bff04d5758a07002ce3b3bf.pkl
# Model in the pickle file: RandomForestClassifier(bootstrap=True, class_weight='balanced',
#            criterion='entropy', max_depth=None, max_features='sqrt',
#            max_leaf_nodes=None, min_impurity_decrease=0.0,
#            min_impurity_split=None, min_samples_leaf=20,
#            min_samples_split=5, min_weight_fraction_leaf=0.0,
#            n_estimators=100, n_jobs=1, oob_score=False, random_state=42,
#            verbose=0, warm_start=False)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
from sklearn.utils import check_X_y
from sklearn.metrics import make_scorer


pickle_file = 'model_5bff04d5758a07002ce3b3bf.pkl'
# file path to the dataset
dataset = 'iris.csv'
# target column name
target_column = 'class'
# seed to be used for train_test_split (default in Aliro is 42)
seed = 42


**NOTE**: Edit variables below with appropriate values.

In [2]:
# path to your pickle file, below is the downloaded pickle file
pickle_file = 'model_5bff04d5758a07002ce3b3bf.pkl'
# file path to the dataset
dataset = 'iris.csv'
# target column name
target_column = 'class'
# seed to be used for train_test_split (default in Aliro is 42)
seed = 42

Balanced accuracy below was described in [Urbanowicz et al. 2015](https://link.springer.com/article/10.1007%2Fs12065-015-0128-8): the average of sensitivity and specificity is computed for each class and then averaged over total number of classes.It is **NOT** the same as [`sklearn.metrics.balanced_accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html), which is defined as the average of recall obtained on each class.

In [3]:
def balanced_accuracy(y_true, y_pred):
    all_classes = list(set(np.append(y_true, y_pred)))
    all_class_accuracies = []
    for this_class in all_classes:
        this_class_sensitivity = 0.
        this_class_specificity = 0.
        if sum(y_true == this_class) != 0:
            this_class_sensitivity = \
                float(sum((y_pred == this_class) & (y_true == this_class))) /\
                float(sum((y_true == this_class)))
            this_class_specificity = \
                float(sum((y_pred != this_class) & (y_true != this_class))) /\
                float(sum((y_true != this_class)))
        this_class_accuracy = (this_class_sensitivity +
                               this_class_specificity) / 2.
        all_class_accuracies.append(this_class_accuracy)
    return np.mean(all_class_accuracies)

## Load fitted model
- load machine learning model from pickle file

In [4]:
pickle_model = joblib.load(pickle_file)
model = pickle_model['model']

## Load input data
- for the application 1, this input dataset should be the same dataset submitted to PennAI
- for the application 2, this input dataset can be a different dataset for testing model of PennAI with holdout score
- for the application 3, this input dataset can be a dataset without target column.

In [5]:
input_data = pd.read_csv(dataset, sep=None, engine='python', dtype=np.float64)

## Application 1: reproducing training score and testing score from PennAI
- This application is to reproduce training score and testing score in PennAI UI with **the same dataset sumbitted to PennAI**.

In [6]:
features = input_data.drop(target_column, axis=1).values
target = input_data[target_column].values
# Checking dataset
features, target = check_X_y(features, target, dtype=np.float64, order="C", force_all_finite=True)
training_features, testing_features, training_classes, testing_classes = \
    train_test_split(features, target, random_state=seed, stratify=input_data[target_column])
scorer = make_scorer(balanced_accuracy)
train_score = scorer(model, training_features, training_classes)
print("Training score: ", train_score)
test_score = scorer(model, testing_features, testing_classes)
print("Testing score: ", test_score)

Training score:  0.9730930930930931
Testing score:  0.9805128205128205


## Application 2: cross validation of fitted model
- This application is to cross validation of fitted model with holdout score.
- The input dataset is a holdout set for calculating holdout score 
- If the holdout score is very different from training score in PennAI, which may mean the model is overfitted.

In [7]:
testing_features = input_data.drop(target_column, axis=1).values
testing_target = input_data[target_column].values
# Get holdout score for fitted model
print("Holdout score: ", end="")
print(model.score(testing_features, testing_target))

Holdout score: 0.9666666666666667


## Application 3: predict outcome by fitted model
- This applicaiton is to predict outcome with fitted model from PennAI
- In this application, the input dataset may not include target column
- Please comment the first line in the cell below if there is no target column in input dataset

In [8]:
input_data.drop(target_column, axis=1, inplace=True) 
predict_target = model.predict(input_data.values)